In [2]:
import requests
import web3
import pandas as pd
import sys
sys.path.append("./chainklik")


import libs.common.utils as utils
import libs.aave.api as aave_api
import libs.common.payload as payload
import libs.common.etherscan as es
import eth_abi
import config.config as cfg

In [3]:
ws = web3.Web3(web3.Web3.HTTPProvider(cfg.config["http_url"]))

In [5]:
ws.eth.get_block("latest")["number"]

18762457

In [6]:
contract_address = "0x7d2768dE32b0b80b7a3454c06BdAc94A69DDc7A9"

In [7]:
proxy_addr = "0x7d2768dE32b0b80b7a3454c06BdAc94A69DDc7A9"
impl_addr = "0x085E34722e04567Df9E6d2c32e82fd74f3342e79"
pool_contract = es.get_contract(proxy_addr,impl_addr)

In [8]:
res = pool_contract.functions.getReservesList().call(block_identifier=18000000)

In [9]:
pool_contract.functions.getUserAccountData("0x1111567E0954E74f6bA7c4732D534e75B81DC42E").call()

[39850552101695585471388,
 32788108131716416302429,
 72657131341763477278,
 8602,
 8246,
 1045484075512107222]

In [12]:
def parse_contract_abi(contract_abi):
    function_abi = {}
    for member in contract_abi:
        if member["type"] == "function":
            inputs = {(inp["name"] if inp["name"] != "" else "value"):inp["type"] for inp in member["inputs"]}
            outputs = {(out["name"] if out["name"] != "" else "value"):out["type"] for out in member["outputs"]}
            function_abi[member["name"]] = {"inputs":inputs,"outputs":outputs}
    return function_abi

In [13]:
def parse_function_abi(function_abi):
    inputs = {(inp["name"] if inp["name"] != "" else "value"):inp["type"] for inp in function_abi["inputs"]}
    outputs = {(out["name"] if out["name"] != "" else "value"):out["type"] for out in function_abi["outputs"]}
    return {"inputs":inputs,"outputs":outputs}

In [14]:
pool_contract.functions.abi

[{'anonymous': False,
  'inputs': [{'indexed': True,
    'internalType': 'address',
    'name': 'reserve',
    'type': 'address'},
   {'indexed': False,
    'internalType': 'address',
    'name': 'user',
    'type': 'address'},
   {'indexed': True,
    'internalType': 'address',
    'name': 'onBehalfOf',
    'type': 'address'},
   {'indexed': False,
    'internalType': 'uint256',
    'name': 'amount',
    'type': 'uint256'},
   {'indexed': False,
    'internalType': 'uint256',
    'name': 'borrowRateMode',
    'type': 'uint256'},
   {'indexed': False,
    'internalType': 'uint256',
    'name': 'borrowRate',
    'type': 'uint256'},
   {'indexed': True,
    'internalType': 'uint16',
    'name': 'referral',
    'type': 'uint16'}],
  'name': 'Borrow',
  'type': 'event'},
 {'anonymous': False,
  'inputs': [{'indexed': True,
    'internalType': 'address',
    'name': 'reserve',
    'type': 'address'},
   {'indexed': False,
    'internalType': 'address',
    'name': 'user',
    'type': 'addre

In [15]:
function_abi = parse_function_abi(pool_contract.functions.abi[28])

In [16]:
contract_abi = parse_contract_abi(pool_contract.functions.abi)

In [17]:
def function_call(contract_address, function_name, function_abi, params, block_number):
    val = payload.func_call_payload(0, contract_address, function_name, function_abi, params, block_number)
    resp = requests.post(cfg.config["http_url"], json=val)
    
    outputs = function_abi["outputs"]
    values = eth_abi.decode(list(outputs.values()), bytes.fromhex(resp.json()["result"][2:]))
    keys = list(outputs.keys())
    result = {keys[i]: values[i] for i in range(len(keys))}
    return result

In [21]:
contract_abi["getUserConfiguration"]

{'inputs': {'user': 'address'}, 'outputs': {'value': 'tuple'}}

In [22]:
function_name = "getUserConfiguration"
params = {
    "user":"0x1111567E0954E74f6bA7c4732D534e75B81DC42E"
}
block_number = hex(18670000)
function_call(contract_address, function_name, contract_abi[function_name], params, block_number)

NoEntriesFound: No matching entries for 'tuple' in decoder registry

In [24]:
val = payload.func_call_payload(0, contract_address, function_name, contract_abi[function_name], params, block_number)

In [26]:
resp = requests.post(cfg.config["http_url"], json=val)

In [30]:
outputs = contract_abi[function_name]["outputs"]
outputs

{'value': 'tuple'}

In [32]:
resp.json()["result"][2:]

'0000000000000000000000000000000000000000000000000000000000000024'

In [33]:
pool_contract.functions.getUserConfiguration("0x1111567E0954E74f6bA7c4732D534e75B81DC42E").call()

(549755813924,)

In [27]:
pool_contract.functions.abi

[{'anonymous': False,
  'inputs': [{'indexed': True,
    'internalType': 'address',
    'name': 'reserve',
    'type': 'address'},
   {'indexed': False,
    'internalType': 'address',
    'name': 'user',
    'type': 'address'},
   {'indexed': True,
    'internalType': 'address',
    'name': 'onBehalfOf',
    'type': 'address'},
   {'indexed': False,
    'internalType': 'uint256',
    'name': 'amount',
    'type': 'uint256'},
   {'indexed': False,
    'internalType': 'uint256',
    'name': 'borrowRateMode',
    'type': 'uint256'},
   {'indexed': False,
    'internalType': 'uint256',
    'name': 'borrowRate',
    'type': 'uint256'},
   {'indexed': True,
    'internalType': 'uint16',
    'name': 'referral',
    'type': 'uint16'}],
  'name': 'Borrow',
  'type': 'event'},
 {'anonymous': False,
  'inputs': [{'indexed': True,
    'internalType': 'address',
    'name': 'reserve',
    'type': 'address'},
   {'indexed': False,
    'internalType': 'address',
    'name': 'user',
    'type': 'addre

In [25]:
function_name = "getReservesList"
params = {}
block_number = hex(18670000)
function_call(function_name, function_abi[function_name], params, block_number)

{'value': ('0xdac17f958d2ee523a2206206994597c13d831ec7',
  '0x2260fac5e5542a773aa44fbcfedf7c193bc2c599',
  '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
  '0x0bc529c00c6401aef6d220be8c6ea1667f6ad93e',
  '0xe41d2489571d322189246dafa5ebde1f4699f498',
  '0x1f9840a85d5af5bf1d1762f925bdaddc4201f984',
  '0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9',
  '0x0d8775f648430679a709e98d2b0cb6250d2887ef',
  '0x4fabb145d64652a948d72533023f6e7a623c7c53',
  '0x6b175474e89094c44da98b954eedeac495271d0f',
  '0xf629cbd94d3791c9250152bd8dfbdf380e2a3b9c',
  '0xdd974d5c2e2928dea5f71b9825b8b646686bd200',
  '0x514910771af9ca656af840dff83e8264ecf986ca',
  '0x0f5d2fb29fb7d3cfee444a200298f468908cc942',
  '0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2',
  '0x408e41876cccdc0f92210600ef50372656052a38',
  '0xc011a73ee8576fb46f5e1c5751ca3b9fe0af2a6f',
  '0x57ab1ec28d129707052df4df418d58a2d46d5f51',
  '0x0000000000085d4780b73119b644ae5ecd22b376',
  '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
  '0xd533a949740bb3306d119cc777